In [1]:
import os
import sys
import numpy as np
import pandas as pd
import torch
from torch_geometric.data import Data
from torch_geometric.data import Batch
from torch_geometric.data import DataLoader
from torch_geometric.nn import TopKPooling
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SAGEConv
from torch.nn import MSELoss, CrossEntropyLoss
from torch_geometric.nn import GCNConv
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import pickle

In [2]:
filehandler = open("../data/sample_data/hotel_reservation_sample.pkl","rb")
dat = pickle.load(filehandler)
filehandler.close()
print(type(dat))
print(dat.shape)

<class 'pandas.core.frame.DataFrame'>
(999996, 7)


In [3]:
global model
model = ""
num_features = 1
num_classes = 6
hc = 32
n_epochs = 20
batch_size = 128

In [4]:
X = dat.drop(columns = "label")
y = dat.loc[:, "label"]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_dat = pd.DataFrame(X_train, columns = ["0_frontend", "1_search", "2_geo", "3_rate", "4_profile", "5_locale"])
train_dat['label'] = y_train.to_list()
test_dat = pd.DataFrame(X_test, columns = ["0_frontend", "1_search", "2_geo", "3_rate", "4_profile", "5_locale"])
test_dat['label'] = y_test.to_list()

In [6]:
X_train.shape
print(len(y[y == 0]))
print(len(y[y == 1]))
print(len(y[y == 2]))
print(len(y[y == 3]))
print(len(y[y == 4]))
print(len(y[y == 5]))


166666
166666
166666
166666
166666
166666


In [7]:
graphs_train = list()
count = 0
for row in train_dat.iterrows():
    r = pd.Series(row)
    r = r[1]
    x = torch.tensor([[r['0_frontend']],
                      [r['1_search']],
                      [r['2_geo']],
                      [r['3_rate']],
                      [r['4_profile']],
                      [r['5_locale']]] ,dtype = torch.float)
    edge_index = torch.tensor([[0, 0, 1, 1, 4],
                               [1, 4, 2, 3, 5]], dtype=torch.long)
    if r['label'] == 0:
        y = torch.tensor([1,0,0,0,0,0], dtype = torch.float)
    elif r['label'] == 1:
        y = torch.tensor([0,1,0,0,0,0], dtype = torch.float)
    elif r['label'] == 2:
        y = torch.tensor([0,0,1,0,0,0], dtype = torch.float)
    elif r['label'] == 3:
        y = torch.tensor([0,0,0,1,0,0], dtype = torch.float)
    elif r['label'] == 4:
        y = torch.tensor([0,0,0,0,1,0], dtype = torch.float)
    elif r['label'] == 5:
        y = torch.tensor([0,0,0,0,0,1], dtype = torch.float)
    else:
        print("unknown label encountered")
        break
    graphs_train.append(Data(x = x, edge_index = edge_index,y = y))
    count = count + 1
    if count % 10000 == 0:
        print(count)
    
count = 0
graphs_test = list()
for row in test_dat.iterrows():
    r = pd.Series(row)
    r = r[1]
    x = torch.tensor([[r['0_frontend']],
                      [r['1_search']],
                      [r['2_geo']],
                      [r['3_rate']],
                      [r['4_profile']],
                      [r['5_locale']]] ,dtype = torch.float)
    edge_index = torch.tensor([[0, 0, 1, 1, 4],
                               [1, 4, 2, 3, 5]], dtype=torch.long)
    if r['label'] == 0:
        y = torch.tensor([1,0,0,0,0,0], dtype = torch.float)
    elif r['label'] == 1:
        y = torch.tensor([0,1,0,0,0,0], dtype = torch.float)
    elif r['label'] == 2:
        y = torch.tensor([0,0,1,0,0,0], dtype = torch.float)
    elif r['label'] == 3:
        y = torch.tensor([0,0,0,1,0,0], dtype = torch.float)
    elif r['label'] == 4:
        y = torch.tensor([0,0,0,0,1,0], dtype = torch.float)
    elif r['label'] == 5:
        y = torch.tensor([0,0,0,0,0,1], dtype = torch.float)
    else:
        print("unknown label encountered")
        break
    graphs_test.append(Data(x = x, edge_index = edge_index,y = y))
    count = count + 1
    if count % 10000 == 0:
        print(count)


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000


In [8]:
print(f'Number of graphs: {len(graphs_train)}')

data = graphs_train[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')

Number of graphs: 799996

Data(edge_index=[2, 5], x=[6, 1], y=[6])
Number of nodes: 6
Number of edges: 5
Average node degree: 0.83


In [9]:
# Initialize scaling and dataloader

scaler = StandardScaler()
loader = DataLoader(graphs_train, batch_size = batch_size)

In [10]:
# class GCN(torch.nn.Module):
#     def __init__(self, hidden_channels):
#         super(GCN, self).__init__()
#         torch.manual_seed(12345)
#         self.conv1 = GCNConv(num_features, hidden_channels)
#         self.conv2 = GCNConv(hidden_channels, num_classes)

#     def forward(self, data):
#         x, edge_index = data.x, data.edge_index
#         x = self.conv1(x, edge_index)
#         x = x.relu()
#         x = F.dropout(x, p=0.3, training=self.training)
#         x = self.conv2(x, edge_index)
# #         return F.softmax(x)
#         return x

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = SAGEConv(num_features, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.1, training=self.training)
        x = self.conv2(x, edge_index)
#         return F.softmax(x)
        return x

In [11]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
for d in range(1):
    data = next(iter(loader))
    data = data.to(device)
    print(data)
print(data.num_node_features)
model = GCN(hidden_channels = hc).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
loss_func = CrossEntropyLoss()   

print(len(graphs_train))
print(len(loader))
print(device)

Batch(batch=[768], edge_index=[2, 640], x=[768, 1], y=[768])
1
799996
6250
cpu


In [ ]:
count = 0
for epoch in range(n_epochs):
    print("ongoing epoch is:", epoch)
    for data in loader:
        data = data.to(device)
        count = count + 1
        optimizer.zero_grad()                                                   
        out = model(data) 
        loss = loss_func(out, data.y.long())
        loss.backward()                                                         
        optimizer.step()
    
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

ongoing epoch is: 0
Epoch: 000, Loss: 0.0283
ongoing epoch is: 1


In [ ]:
num_pred = 0
count_pred = 0
count = 0
for tg in graphs_test:
    out = model(tg)
    pred = out.argmax(dim=1)
    if torch.all(torch.eq(tg.y, pred)):
        num_pred += 1
#         print(pred, tg.y)
    count_pred +=1
    count += 1

    if count % 10000 == 0:
        print(count)

print("Accuracy: ", num_pred/count_pred)